**Purpose:** Investigate synchrony between the cell division and metabolic cycles.

This is inspired by fig 1 of Bieler et al. (2014), show below.  It shows the relationship between the cell division and circadian cycles.

![title](msb0010-0739-f1.jpg)

Figure 1. Circadian and cell cycle oscillators are tightly synchronized in NIH3T3 cells
- A. Single-cell time traces showing the circadian YFP signal (black, identified maxima in blue denoted as p), together with cell division events (nuclear envelope breakdown, red, denoted as d). The top trace is typical and shows three divisions before the circadian peaks, the second trace shows an early first division.
- B. Raster plot showing 3,160 traces (with at least two circadian peaks) aligned on the second circadian peak (blue), and sorted according to the interval between the first and second circadian peaks. Divisions (red) show a clear tendency to occur, on average, 5 h before the circadian peaks. A sparse group of early division events associated usually with longer circadian intervals is also visible.
- C. Division times measured with respect to the subsequent circadian peak show a unimodal distribution centered at −5 h. Inset: longer circadian intervals correlate with mitosis occurring, on average, closer to the next peak (also visible in B).
- D. Circadian phases at division (normalized division times) show a unimodal distribution. Inset: longer circadian intervals correlate with mitosis occurring at later circadian phases.

**Aims:**
- Import flavin signals and birth events from a population of same-strain cells subject to the same nutrient conditions.
    - Start from the more reliable BY4741 or FY4 time series first, to iron out any issues.
    - Real purpose is to do this with the mutants, especially with the `swe1_Del` from the Causton lab.  It's only going to be a matter of switching the dataset file.
- Process data: cut time series to duration of interest (births matrix should be cut too), detrend flavin signals.
- Align time series to the first birth event.
- Locate the subsequent metabolic cycle.
    - This may be through finding a peak, using the appropriate tolerance.
    - Fitting an autoregressive model or Gaussian process may be helpful.
    - I will also need to identify the troughs so that I only get one metabolic cycle (see heatmap in next main item).
    - I expect this to be the most technically difficult.  If all else fails, plotting a heatmap with birth times overlaid 'for now' may still help produce insight.
- Plot a heatmap that shows:
    - Where the second birth event is in relation to the first birth event.
    - Where the subsequent metabolic cycle is in relation to the first metabolic cycle.
- Plot histograms that show the distributions of:
    - Time differences between first and second birth events.
    - Time differences between first birth event and peak of metabolic cycle.

**Paradigms:**
- Use `aliby` data structures, i.e. `pandas` `DataFrames` with multi-indexing.
    - Births has only been recently (2022-01-18) added to the `DataFrame` output.  Need to look at how it is done there first, and try to replicate.
- Use `postprocessor` processes e.g. `fft` (Fourier transform), `autoreg` (autoregressive model -- add methods as appropriate).
- Ultimate goal to put all the cells together in a script to put in `skeletons`, but some people prioritise having plots fast over having clean code.

In [ ]:
%matplotlib inline

# Import data

**Important note: must be mindful of how I'm dealing with TWO sets of signals -- flavin and births.  Current code, copied from `svm_sandbox.ipynb`, deals with ONE set of signals; having two sets is another can of worms.  Name the data variables and manage them wisely.**

In [ ]:
import numpy as np
import pandas as pd
import csv

# PARAMETERS
#filename_prefix = './data/arin/Omero19979_'
filename_prefix = './data/arin/Omero20016_'
#

# Import flavin signals
signal_flavin = pd.read_csv(filename_prefix+'flavin.csv')
signal_flavin.replace(0, np.nan, inplace=True) # because the CSV is constructed like that :/

# Import birth signals
signal_births = pd.read_csv(filename_prefix+'births.csv')
#signal_births.replace(0, np.nan, inplace=True)

def convert_df_to_aliby(
    signal,
    strainlookup_df,
):
    # Import look-up table for strains (would prefer to directly CSV -> dict)
    strainlookup_dict = dict(zip(strainlookup_df.position, strainlookup_df.strain))
    
    # Positions -> Strain (more informative)
    signal = signal.replace({'position': strainlookup_dict})
    signal.rename(columns = {"position": "strain"}, inplace = True)
    signal = signal.drop(['distfromcentre'], axis = 1)

    # Convert to multi-index dataframe
    signal_temp = signal.iloc[:,2:]
    multiindex = pd.MultiIndex.from_frame(signal[['strain', 'cellID']])
    signal = pd.DataFrame(signal_temp.to_numpy(),
                          index = multiindex)
    
    return signal

strainlookup_df = pd.read_csv(filename_prefix+'strains.csv')
signal_flavin = convert_df_to_aliby(signal_flavin, strainlookup_df)
signal_births = convert_df_to_aliby(signal_births, strainlookup_df)

In [ ]:
signal_flavin

In [ ]:
signal_births

# Choose a list of cells as working data

List strains

In [ ]:
signal_flavin.index.get_level_values(0).unique().to_list()

Define `signal_flavin_wd` as working data

In [ ]:
strain = 'by4741'

signal_flavin_wd = signal_flavin.loc[strain]
signal_births_wd = signal_births.loc[strain]

# Processing time series

## Range

Chop up time series according to `interval_start` and `interval_end`, then remove cells that have NaNs.  Print number of cells.

In [ ]:
# PARAMETERS
interval_start = 25
interval_end = 168
#

signal_flavin_processed = signal_flavin_wd.iloc[:, interval_start:interval_end]
# note: originally had dropna() at the end -- it should be removed later, but I will have to match flavin and births by cellID
signal_births_processed = signal_births_wd.iloc[:, interval_start:interval_end]
signal_flavin_processed

In [ ]:
import matplotlib.pyplot as plt
plt.plot(signal_flavin_processed.iloc[3])

## Detrend

Using sliding window

In [ ]:
# POTENTIAL ISSUE: This removes some time points.  Need to find a way to re-align births so that it makes sense.
# I expect that there is a constant shift that is a function of the size of the sliding window.
# ----> seems like no re-aligning needed???  After a cursory look at some time series & where their peaks should be.

import matplotlib.pyplot as plt
import seaborn as sns

# PARAMETERS
window = 45
#

fig, ax = plt.subplots()
sns.heatmap(signal_flavin_processed)
plt.title('Before detrending')
plt.show()

def moving_average(input_timeseries,
                  window = 3):
    processed_timeseries = np.cumsum(input_timeseries, dtype=float)
    processed_timeseries[window:] = processed_timeseries[window:] - processed_timeseries[:-window]
    return processed_timeseries[window - 1 :] /  window

signal_flavin_processed = signal_flavin_processed.div(signal_flavin_processed.mean(axis = 1), axis = 0)
signal_flavin_movavg = signal_flavin_processed.apply(lambda x: pd.Series(moving_average(x.values, window)), axis = 1)
signal_flavin_norm = signal_flavin_processed.iloc(axis = 1)[window//2: -window//2] / signal_flavin_movavg.iloc[:,0:signal_flavin_movavg.shape[1]-1].values

fig, ax = plt.subplots()
sns.heatmap(signal_flavin_norm)
plt.title('After detrending')
plt.show()

signal_flavin_processed = signal_flavin_norm
signal_births_processed = signal_births_processed.iloc[:, :-window]

signal_flavin_processed

In [ ]:
signal_births_processed

In [ ]:
plt.plot(signal_flavin_processed.iloc[3])

## Align to first birth

In [ ]:
# For each cell:
# - Identify the first birth.
# - In the births signal: delete all that comes before it; make it time point 0.
# - In the corresponding flavin signal: delete all that comes before the first birth;
#   make the corresponding flavin time point 0.
# - Fill the rest of the dataframe with NaNs?  To ensure rectangular dimensions.

In [ ]:
signal_flavin_save = signal_flavin_processed
signal_flavin_save

In [ ]:
signal_flavin_aligned = signal_flavin_save

shifted_rows = []
for row_index in range(len(signal_flavin_aligned)):
    birth_locs = np.where(signal_births_processed.iloc[row_index].to_numpy() == 1)[0]
    if birth_locs.any():
        shift = birth_locs[0]
    else:
        shift = 0
    shifted_rows.insert(
        row_index,
        signal_flavin_aligned.iloc[[row_index]].shift(periods = -shift, axis = 'columns')
    )
signal_flavin_aligned = pd.concat(shifted_rows)
signal_flavin_aligned

# Locate metabolic cycle peaks

In [ ]:
# (add code here)

# Plot heatmaps

In [ ]:
# Should be straightforward with seaborn.heatmap().
# It's essentially a heatmap from the birth signals and another heatmap from the flavin signals.
# I just have to find a way to superimpose them.
import seaborn as sns
import matplotlib.pyplot as plt
plt.imshow(signal_births_processed)

In [ ]:
plt.imshow(signal_flavin_aligned)

# Plot histograms

In [ ]:
# - From birth signals: find duration between 1st and 2nd birth (should just be an easy subtraction problem)
# - From flavin signals: find duration between peak (which should already be found earlier)
#   and the corresponding 1st birth.
# - When these durations are found, it should be straightforward with matplotlib.pyplot.hist()